In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import seaborn as sns

In [ ]:
train=pd.read_csv("/kaggle/input/mnist-in-csv/mnist_train.csv")
print(train.shape)
train.head()

In [ ]:
test=pd.read_csv("/kaggle/input/mnist-in-csv/mnist_test.csv")
print(test.shape)
test.head()

In [ ]:
X_mnist_train=train.drop(labels=["label"],axis=1)
y_mnist_train=train["label"]

In [ ]:
X_mnist_test=test.drop(labels=["label"],axis=1)
y_mnist_test=test["label"]

In [ ]:
X_mnist_full = np.concatenate((X_mnist_train, X_mnist_test))
y_mnist_full = np.concatenate((y_mnist_train, y_mnist_test))

In [ ]:
np.unique(y_mnist_full).shape

In [ ]:
img = X_mnist_train.iloc[3000].values.reshape(28,28)
img.shape
plt.imshow(img,cmap='gray')
plt.axis("off")
plt.show()

In [ ]:
print(np.unique(y_mnist_full))

In [ ]:
permutation = np.random.permutation(X_mnist_full.shape[0])

X_mnist_full = X_mnist_full[permutation]
y_mnist_full = y_mnist_full[permutation]

In [ ]:
X_mnist_full = X_mnist_full / 255.0

In [ ]:
import string

unique_values, frequency_count = np.unique(y_mnist_full, return_counts=True)
label = ["%d" %i for i in range(0,10)]
plt.bar(label, frequency_count)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_mnist_full, y_mnist_full, test_size=0.1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.11)

In [ ]:
np.unique(y_train).shape

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')
X_valid = X_valid.reshape(X_valid.shape[0], 28, 28, 1).astype('float32')

y_train_cat = keras.utils.to_categorical(y_train)
y_test_cat = keras.utils.to_categorical(y_test)
y_valid_cat = keras.utils.to_categorical(y_valid)

In [ ]:
y_train_cat.shape[1]

In [ ]:
print(y_train[3009])
plt.imshow(X_train[3009], cmap='gray')
plt.show()

**CNN MODEL**

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=64, kernel_size=(7,7), padding='valid', activation='relu'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train,y_train_cat , epochs=40,  validation_data = (X_valid,y_valid_cat))

In [ ]:
q = len(history.history['accuracy'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+q),y = history.history['accuracy'], label='Train_Accuracy')
sns.lineplot(x = range(1,1+q),y = history.history['val_accuracy'], label='Val_Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

**Evaluating Model on Test Dataset**

In [ ]:
model.evaluate(X_test, y_test_cat)